In [1]:
import os
import shutil
from tqdm import tqdm

In [2]:
train_folder = "/Users/monicagullapalli/Downloads/emotion_detection/train"
validation_folder = "/Users/monicagullapalli/Downloads/emotion_detection/validation"

In [3]:
combined_folder = "//Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/combined"

In [4]:
def copy_files_with_labels(source_folder, destination_folder):
    for label in os.listdir(source_folder):
        label_folder = os.path.join(source_folder, label)
        if os.path.isdir(label_folder):
            destination_label_folder = os.path.join(destination_folder, label)
            if not os.path.exists(destination_label_folder):
                os.makedirs(destination_label_folder)
            for file in tqdm(os.listdir(label_folder), desc=f"Copying {label}"):
                shutil.copy(os.path.join(label_folder, file), destination_label_folder)


copy_files_with_labels(train_folder, combined_folder)


copy_files_with_labels(validation_folder, combined_folder)

print("Data combined successfully.")

Copying Happy:   0%|          | 0/7215 [00:00<?, ?it/s]

Copying Disgust: 100%|██████████| 55/55 [00:00<00:00, 3769.14it/s]

Data combined successfully.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax') 
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    '/Users/monicagullapalli/Downloads/emotion_detection/train',
    target_size=(64, 64), 
    batch_size=32,
    class_mode='sparse'  
)


validation_generator = datagen.flow_from_directory(
    '/Users/monicagullapalli/Downloads/emotion_detection/validation',
    target_size=(64, 64), 
    batch_size=32,
    class_mode='sparse' 
)


model.fit(train_generator, validation_data=validation_generator, epochs=10)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Epoch 1/10
898/898 [==============================] - 32s 35ms/step - loss: 1.6363 - accuracy: 0.3660 - val_loss: 1.5900 - val_accuracy: 0.3795
Epoch 2/10
898/898 [==============================] - 34s 38ms/step - loss: 1.4143 - accuracy: 0.4589 - val_loss: 1.5684 - val_accuracy: 0.3976
Epoch 3/10
898/898 [==============================] - 34s 37ms/step - loss: 1.2802 - accuracy: 0.5150 - val_loss: 1.5849 - val_accuracy: 0.4199
Epoch 4/10
898/898 [==============================] - 36s 40ms/step - loss: 1.1543 - accuracy: 0.5681 - val_loss: 1.6672 - val_accuracy: 0.4129
Epoch 5/10
898/898 [==============================] - 35s 39ms/step - loss: 1.0139 - accuracy: 0.6247 - val_loss: 1.7938 - val_accuracy: 0.4227
Epoch 6/10
898/898 [==============================] - 34s 38ms/step - loss: 0.8762 - accuracy: 0.6825 - val_loss: 1.9179 - val_accuracy: 0.4238
Epoch 7/10
898/898 [==============================]

In [6]:
model.save('/Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/final_model.h5')

/Users/monicagullapalli/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
import os
from tqdm import tqdm
from PIL import Image

# Function to classify all images in a folder using VGG16
def classify_images_vgg16(folder_path):
    for filename in tqdm(os.listdir(folder_path)):
        image_path = os.path.join(folder_path, filename)
        if os.path.isfile(image_path):
            try:
                Image.open(image_path)
            except (IOError, OSError):
                print(f"Skipping non-image file: {image_path}")
                continue

            print(f"Classifying image: {image_path}")
            classify_image_vgg16(image_path)

# Provide the path to the train folder
train_folder = "/Users/monicagullapalli/Downloads/emotion_detection/train"

# Classify all images in the train folder using VGG16
classify_images_vgg16(train_folder)

100%|██████████| 8/8 [00:00<00:00, 90.66it/s]

Skipping non-image file: /Users/monicagullapalli/Downloads/emotion_detection/train/.DS_Store


In [10]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tqdm import tqdm
from PIL import Image

# Provide the path to the train folder
train_folder = "/Users/monicagullapalli/Downloads/emotion_detection/train"

# Set the number of epochs
num_epochs = 10

# Define the loss function
loss_fn = CategoricalCrossentropy()

# Create the base VGG16 model
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers
base_model.trainable = False

# Add a custom output layer with 7 units for the emotion classes
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(7, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
optimizer = Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Run the training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0

    for emotion_folder in os.listdir(train_folder):
        subfolder_path = os.path.join(train_folder, emotion_folder)
        if not os.path.isdir(subfolder_path):
            continue

        for filename in tqdm(os.listdir(subfolder_path)):
            image_path = os.path.join(subfolder_path, filename)
            if os.path.isfile(image_path):
                try:
                    Image.open(image_path)
                except (IOError, OSError):
                    print(f"Skipping non-image file: {image_path}")
                    continue

                img = image.load_img(image_path, target_size=(224, 224))
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)

                # Assuming you have labels for each image, you can load and preprocess them here
                emotion_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
                label = np.zeros((1, len(emotion_classes)))
                label[0, emotion_classes.index(emotion_folder)] = 1

                # Perform training on the image batch
                with tf.GradientTape() as tape:
                    logits = model(x, training=True)
                    loss_value = loss_fn(label, logits)

                grads = tape.gradient(loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))

                # Update loss and accuracy
                total_loss += loss_value
                total_accuracy += np.mean(np.argmax(logits, axis=1) == np.argmax(label, axis=1))
                num_batches += 1

    # Calculate average loss and accuracy for the epoch
    if num_batches > 0:
        average_loss = total_loss / num_batches
        average_accuracy = total_accuracy / num_batches
        print(f"Loss: {average_loss:.4f} - Accuracy: {average_accuracy:.4f}")
    else:
        print("No valid image files found.")

    # Save the model weights after each epoch
    model.save_weights(f"vgg16_epoch_{epoch + 1}.h5")

Epoch 1/10


100%|██████████| 436/436 [20:12<00:00,  2.78s/it]   


Loss: 0.0672 - Accuracy: 0.9974
Epoch 2/10


 70%|██████▉   | 2219/3171 [49:11<21:47,  1.37s/it]  

In [ ]:
from PIL import Image, ImageTk

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import webbrowser
import tkinter as tk
from PIL import Image, ImageTk

# Load the pre-trained emotion classification model
model = load_model('/Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/final_model.h5')

class_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}

# Define Spotify playlist URLs for each emotion
spotify_playlists = {
    'Angry': 'https://spotify.com/playlist/angry_playlist',
    'Sad': 'https://spotify.com/playlist/sad_playlist',
    'Happy': 'https://open.spotify.com/playlist/049OGKQgNXgRRJh91zNwi7',
    'Disgust': 'https://spotify.com/playlist/disgust_playlist',
    'Fear': 'https://spotify.com/playlist/fear_playlist',
    'Neutral': 'https://spotify.com/playlist/neutral_playlist',
    'Surprise': 'https://spotify.com/playlist/surprise_playlist'
}

# Initialize the camera
cap = cv2.VideoCapture(0)

# Function to capture image and perform actions
def capture_image():
    ret, frame = cap.read()
    processed_frame = cv2.resize(frame, (64, 64))
    processed_frame = processed_frame / 255.0

    input_data = np.expand_dims(processed_frame, axis=0)

    predictions = model.predict(input_data)
    predicted_class = np.argmax(predictions)
    predicted_label = class_labels[predicted_class]

    # Perform actions based on the predicted class label
    if predicted_label == 'Angry':
        # Redirect to Angry playlist on Spotify
        webbrowser.open(spotify_playlists['Angry'])

    elif predicted_label == 'Sad':
        # Redirect to Sad playlist on Spotify
        webbrowser.open(spotify_playlists['Sad'])

    elif predicted_label == 'Happy':
        # Redirect to Happy playlist on Spotify
        webbrowser.open(spotify_playlists['Happy'])

    elif predicted_label == 'Disgust':
        # Redirect to Disgust playlist on Spotify
        webbrowser.open(spotify_playlists['Disgust'])

    elif predicted_label == 'Fear':
        # Redirect to Fear playlist on Spotify
        webbrowser.open(spotify_playlists['Fear'])

    elif predicted_label == 'Neutral':
        # Redirect to Neutral playlist on Spotify
        webbrowser.open(spotify_playlists['Neutral'])

    elif predicted_label == 'Surprise':
        # Redirect to Surprise playlist on Spotify
        webbrowser.open(spotify_playlists['Surprise'])

# Create a Tkinter window
window = tk.Tk()

# Function to capture frame and perform actions
def capture_frame():
    # Capture a frame from the camera
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame)

    # Display the captured frame
    image.show()

    # Perform actions on the captured frame
    capture_image()

# Create a capture button
capture_button = tk.Button(window, text='Capture', command=capture_frame)
capture_button.pack()

# Start the Tkinter event loop
window.mainloop()

# Release the camera
cap.release()
cv2.destroyAllWindows()
